# Imports

In [ ]:
import itertools as it
import os
import re
from collections import namedtuple
from functools import partial
from pathlib import Path

import dask
import distributed
import h5py
import holoviews as hv
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
import skimage.measure
import zarr
from dask import delayed
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster, progress
from tqdm.auto import tqdm, trange

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from paulssonlab.image_analysis import *
from paulssonlab.util.ui import display_image

In [ ]:
%load_ext pyinstrument

In [ ]:
hv.extension("bokeh")

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="06:00:00",
    memory="2GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/log",
    cores=1,
    processes=1,
)
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(1)

In [ ]:
cluster.adapt(maximum=20)

# Trench detection

In [ ]:
# filename = "/home/jqs1/scratch/jqs1/microscopy/230213/230213induction.nd2"
# filename = "/home/jqs1/scratch/jqs1/microscopy/230215/230215induction.nd2" #v=7
# filename = "/home/jqs1/scratch/jqs1/microscopy/230326/230326promrbs.nd2" #v=8,t=10
filename = "/home/jqs1/scratch/jqs1/microscopy/230404/230404_rbsprom.nd2"

In [ ]:
nd2 = nd2reader.ND2Reader(filename)

In [ ]:
nd2.metadata["channels"]

In [ ]:
nd2.sizes

In [ ]:
img = nd2.get_frame_2D(v=8, c=0, t=0)

In [ ]:
display_image(img, scale=0.9)

In [ ]:
display_image(img, scale=0.99)

# Radial distortion correction

In [ ]:
k1 = 8.947368421052635e-10
# k1 = 2e-8

In [ ]:
%%time
img_t = image.correct_radial_distortion(img, k1=k1)

In [ ]:
%%time
diag = util.tree()
trenches, info = trench_detection.find_trenches(
    img_t, width_to_pitch_ratio=1.4 / 3.5, join_info=False, diagnostics=diag
)

In [ ]:
info

In [ ]:
diag["bboxes"]

# Radial distortion optimization

In [ ]:
k1s = np.linspace(-1e-9, 2e-9, 20)
res = []
for k1 in tqdm(k1s):
    img_corrected = image.correct_radial_distortion(img, k1=k1)
    h, theta, rho = trench_detection.hough.hough_line_intensity(
        img_corrected, theta=np.linspace(-np.pi / 50, np.pi / 50, 400)
    )
    smooth = 4
    diff_h = np.diff(h.astype(np.int_), axis=1)  # TODO: is diff necessary??
    diff_h_std = diff_h.std(axis=0)  # / diff_h.max(axis=0)
    if smooth:
        diff_h_std_smoothed = scipy.ndimage.gaussian_filter1d(diff_h_std, smooth)
    else:
        diff_h_std_smoothed = diff_h_std
    theta_idx = diff_h_std_smoothed.argmax()
    diff_h_std_max = diff_h_std_smoothed[theta_idx]
    angle = theta[theta_idx]
    res.append(
        dict(
            k1=k1,
            h=h,
            diff_h=diff_h,
            diff_h_std=diff_h_std,
            diff_h_std_smoothed=diff_h_std_smoothed,
            angle=angle,
            theta_idx=theta_idx,
            img_corrected=img_corrected,
        )
    )

In [ ]:
plt.figure(figsize=(20, 20))
for idx in range(len(res)):
    plt.plot(res[idx]["diff_h_std"][120:200], label=idx)
plt.legend()
plt.plot(res[12]["diff_h_std"][120:200], lw=4)

In [ ]:
plt.figure(figsize=(20, 20))
idx = 12
plt.plot(res[idx - 2]["diff_h_std"][120:200])
plt.plot(res[idx]["diff_h_std"][120:200], lw=4)
plt.plot(res[idx + 2]["diff_h_std"][120:200])

In [ ]:
res[12]["k1"]

In [ ]:
display_image(res[-3]["img_corrected"], scale=True)

# FISH correction

In [ ]:
# filename = "/home/jqs1/scratch/jqs1/microscopy/230213/230213induction.nd2"
# filename = "/home/jqs1/scratch/jqs1/microscopy/230215/230215induction.nd2" #v=7
# filename = "/home/jqs1/scratch/jqs1/microscopy/230326/230326promrbs.nd2" #v=8,t=10
filename = "/home/jqs1/scratch/jqs1/microscopy/230404/230404_rbsprom.nd2"
fish_filename = Path(filename).parent / "FISH/real_run"

In [ ]:
k1 = 8.947368421052635e-10

In [ ]:
def calibrate_image(img, k1=0):
    img = skimage.img_as_float32(img)
    img = image.correct_radial_distortion(img, k1=k1)
    return img

In [ ]:
%%time
delayed = util.get_delayed(True)
fish_frames = {}
fish_crops = {}
fish_channels = set()
fish_timepoints = set()
for msg in readers.send_eaton_fish(
    fish_filename,
    r"fov=(?P<v>\d+)_config=(?P<c>\w+)_t=(?P<t>\d+)",
    slices=dict(t=None, v=[8]),
    delayed=delayed,
):
    # print(msg["metadata"],msg["image"].shape)
    fish_img = msg["image"]
    # fish_img_corrected = delayed(calibrate_image)(fish_img, k1=k1)
    fish_img_corrected = delayed(skimage.img_as_float32)(fish_img)
    t = msg["metadata"]["t"]
    channel = msg["metadata"]["channel"]
    fish_channels.add(channel)
    fish_timepoints.add(t)
    fish_frames[(t, channel)] = fish_img_corrected
fish_channels = list(sorted(fish_channels))
fish_timepoints = list(sorted(fish_timepoints))

In [ ]:
fish_frames0 = dask.compute(fish_frames)[0]

In [ ]:
%%time
stacks = {}
for channel in fish_channels:
    for timepoint_idx, timepoint in enumerate(fish_timepoints):
        img = fish_frames0[(timepoint, channel)]
        if channel not in stacks:
            stacks[channel] = np.full((len(fish_timepoints), *img.shape), np.nan)
        stacks[channel][timepoint_idx, :, :] = img

In [ ]:
stacks["GFP"].shape

In [ ]:
stacks["GFP"][3:9].max(axis=0)

In [ ]:
stacks["GFP"][3:9].max(axis=0)

In [ ]:
display_image(stacks["GFP"][3:9].max(axis=0), scale=0.99)

In [ ]:
display_image(stacks["GFP"][3:9].min(axis=0), scale=0.99)

In [ ]:
display_image(
    stacks["GFP"][3:9].max(axis=0) - stacks["GFP"][3:9].min(axis=0), scale=0.99
)

In [ ]:
display_image(stacks["GFP"][:9].max(axis=0) - stacks["GFP"][:9].min(axis=0), scale=0.99)

In [ ]:
info

# Drift correction

In [ ]:
def get_crop(img, trenches, trench_idx):
    ul_x = trenches["ul_x"].values
    ul_y = trenches["ul_y"].values
    lr_x = trenches["lr_x"].values
    lr_y = trenches["lr_y"].values
    return img[
        ul_y[trench_idx] : lr_y[trench_idx] + 1, ul_x[trench_idx] : lr_x[trench_idx] + 1
    ]

In [ ]:
imgs = {t: nd2.get_frame_2D(v=8, c=0, t=t)[:500, :500] for t in trange(225)}

In [ ]:
hv.HoloMap({k: ui.RevImage(v) for k, v in imgs.items()})

In [ ]:
%%time
diag = util.tree()
trenches, info = trench_detection.find_trenches(
    imgs[0], width_to_pitch_ratio=1.4 / 3.5, join_info=False, diagnostics=diag
)

In [ ]:
trenches

In [ ]:
diag["bboxes"]

In [ ]:
%%time
diag2 = util.tree()
trenches2, info2 = trench_detection.find_trenches(
    imgs[20], width_to_pitch_ratio=1.4 / 3.5, join_info=False, diagnostics=diag2
)

In [ ]:
diag2["bboxes"]

In [ ]:
%%time
diag3 = util.tree()
trenches3, info3 = trench_detection.find_trenches(
    imgs[210], width_to_pitch_ratio=1.4 / 3.5, join_info=False, diagnostics=diag3
)

In [ ]:
diag3["bboxes"]

In [ ]:
ui.RevImage(imgs[210]) * trench_detection.plot_trenches(trenches2)

In [ ]:
ui.RevImage(imgs[210]) * trench_detection.plot_trenches(trenches)

In [ ]:
trench_detection.plot_trenches(trenches2).opts(
    hv.opts.Rectangles(line_color="blue")
) * trench_detection.plot_trenches(trenches3)

In [ ]:
c = get_crop(imgs[20], trenches2, 10).T
c

In [ ]:
plt.imshow(image.hessian_eigenvalues(c)[0])

In [ ]:
?misc.holoborodko_diff.holo_diff

In [ ]:
plt.plot(misc.holoborodko_diff.holo_diff(1, c[5], 4))

In [ ]:
plt.plot(c[5])
# plt.plot(np.diff(c[5]))
plt.plot(misc.holoborodko_diff.holo_diff(1, c[5], 4))

In [ ]:
misc.holoborodko_diff.holo_diff(
    1, scipy.ndimage.gaussian_filter1d(c[5].astype(np.float32), 2), 4
)

In [ ]:
d = misc.holoborodko_diff.holo_diff(
    1, scipy.ndimage.gaussian_filter1d(c.mean(axis=0).astype(np.float32), 2), 2
)  # **2
# d = misc.holoborodko_diff.holo_diff(1, scipy.ndimage.gaussian_filter1d(c[5].astype(np.float32), 2), 2)
# d = misc.holoborodko_diff.holo_diff(1, c[5].astype(np.float64), 2)
plt.plot(c[5])
# plt.plot(np.diff(c[5]))
plt.plot(d * 10)

In [ ]:
np.asarray(skimage.img_as_float(get_crop(imgs[20], trenches2, 10)))

In [ ]:
scipy.signal.find_peaks(d, rel_height=0.5, width=None, prominence=None)

In [ ]:
?scipy.signal.find_peaks

In [ ]:
scipy.signal.find_peaks(
    d,
    height=0.003,  # distance=None,
    # prominence=None,
    # wlen=None,
    width=(None, None),
)

In [ ]:
import warnings


def trench_cell_endpoints2(img, sigma=2, min_height=0.3, margin_factor=1):
    img = skimage.img_as_float(img)
    grad = misc.holoborodko_diff.holo_diff(
        1, scipy.ndimage.gaussian_filter1d(img.mean(axis=1), sigma)
    )
    with warnings.catch_warnings(
        action="ignore", category=scipy.signal._peak_finding_utils.PeakPropertyWarning
    ):
        pos_peaks, pos_peak_props = scipy.signal.find_peaks(
            grad, height=min_height * grad.max(), width=(None, None)
        )
        neg_peaks, neg_peak_props = scipy.signal.find_peaks(
            -grad, height=-min_height * grad.min(), width=(None, None)
        )
    y1 = pos_peaks[0]
    y2 = neg_peaks[-1]
    margin = int(
        np.ceil(
            margin_factor
            * (pos_peak_props["widths"][0] + neg_peak_props["widths"][-1])
            / 2
        )
    )
    cutoff1 = min(y1 + 1 + margin, img.shape[0])
    cutoff2 = max(y2 - margin, 0)
    weights1 = grad[:cutoff1, np.newaxis]
    weights2 = -grad[cutoff2:, np.newaxis]
    x1 = (
        np.arange(img.shape[1])[np.newaxis, :] * img[:cutoff1, :] ** 2 * weights1
    ).sum() / (img[:cutoff1, :] ** 2 * weights1).sum()
    # x1 = (np.arange(img.shape[1])[np.newaxis,:] * img[:cutoff1,:]**2).sum() / (img[:cutoff1,:]**2).sum()
    x2 = (
        np.arange(img.shape[1])[np.newaxis, :] * img[cutoff2:, :] ** 2 * weights2
    ).sum() / (img[cutoff2:, :] ** 2 * weights2).sum()
    # x2 = (np.arange(img.shape[1])[np.newaxis,:] * img[cutoff2:,:]**2).sum() / (img[cutoff2:,:]**2).sum()
    return np.array([[x1, y1], [x2, y2]])

In [ ]:
def trench_cell_endpoints(img, sigma=2, k=2, min_height=0.3, margin_factor=1):
    img = skimage.img_as_float(img)
    profile = img.mean(axis=1)
    grad = misc.holoborodko_diff.holo_diff(
        1, scipy.ndimage.gaussian_filter1d(profile, sigma)
    )
    with warnings.catch_warnings(
        action="ignore", category=scipy.signal._peak_finding_utils.PeakPropertyWarning
    ):
        pos_peaks, pos_peak_props = scipy.signal.find_peaks(
            grad, height=min_height * grad.max(), width=(None, None)
        )
        neg_peaks, neg_peak_props = scipy.signal.find_peaks(
            -grad, height=-min_height * grad.min(), width=(None, None)
        )
    y1 = pos_peaks[0]
    y2 = neg_peaks[-1]
    margin = int(
        np.ceil(
            margin_factor
            * (pos_peak_props["widths"][0] + neg_peak_props["widths"][-1])
            / 2
        )
    )
    cutoff1 = min(y1 + 1 + margin, img.shape[0])
    cutoff2 = max(y2 - margin, 0)
    weights1 = profile[:cutoff1, np.newaxis]
    weights2 = profile[cutoff2:, np.newaxis]
    x1 = (
        np.arange(img.shape[1])[np.newaxis, :] * (img[:cutoff1, :] * weights1) ** k
    ).sum() / ((img[:cutoff1, :] * weights1) ** k).sum()
    x2 = (
        np.arange(img.shape[1])[np.newaxis, :] * (img[cutoff2:, :] * weights2) ** k
    ).sum() / ((img[cutoff2:, :] * weights2) ** k).sum()
    return np.array([[x1, y1], [x2, y2]])

In [ ]:
crop.shape

In [ ]:
plt.imshow(crop[:, ::-1])

In [ ]:
pts = trench_cell_endpoints(crop[:, ::1])
pts

In [ ]:
grad = misc.holoborodko_diff.holo_diff(
    1, scipy.ndimage.gaussian_filter1d(skimage.img_as_float(crop).mean(axis=1), 2)
)
scipy.signal.find_peaks(-grad, height=0.3 * grad.max(), width=(None, None))

In [ ]:
# idx=59
idx = 80
crop = get_crop(imgs[idx], trenches2, 10)
pts = trench_cell_endpoints(crop)
ui.RevImage(crop).opts(frame_width=40) * hv.Points(pts + 0.5).opts(color="red", size=4)
# hv.Points(pts)

In [ ]:
# idx=59
idx = 89
crop = get_crop(imgs[idx], trenches2, 10)
pts = trench_cell_endpoints(crop)
ui.RevImage(crop).opts(frame_width=40) * hv.Points(pts + 0.5).opts(color="red", size=4)
# hv.Points(pts)

In [ ]:
plots = {}
for t in trange(225):
    crop = get_crop(imgs[t], trenches2, 10)
    pts = trench_cell_endpoints(crop)
    plots[t] = ui.RevImage(crop).opts(frame_width=40) * hv.Points(pts + 0.5).opts(
        color="red", size=4
    )

In [ ]:
hv.HoloMap(plots)

In [ ]:
np.arange(crop.shape[1])[np.newaxis, :].mean()

In [ ]:
(np.arange(crop.shape[1])[np.newaxis, :] * crop[:42]).mean() / crop[:42].mean()

In [ ]:
(np.arange(crop.shape[1])[np.newaxis, :] * crop[:42] * d[:42, np.newaxis]).sum() / crop[
    :42
].sum() / d[:42, np.newaxis].sum()

In [ ]:
(np.arange(crop.shape[1])[np.newaxis, :] * crop[:42] * d[:42, np.newaxis]).mean() / (
    d[:42, np.newaxis] * crop[:42]
).mean()

In [ ]:
(crop[41] * np.arange(crop.shape[1])).mean() / crop[41].mean()

In [ ]:
plt.plot(crop[41])

In [ ]:
plt.imshow(np.arange(crop.shape[1])[np.newaxis, :] * crop[:42] * d[:42, np.newaxis]);

In [ ]:
plt.imshow(np.arange(crop.shape[1])[np.newaxis, :] * crop[:42]);

In [ ]:
plt.imshow(crop[:42]);

In [ ]:
(
    (np.arange(img.shape[1]) + 0.5)[np.newaxis, :]
    * img[y2:, :]
    * weights2[:, np.newaxis]
).mean() / (img[y2:, :].mean() * weights2.mean())

In [ ]:
scipy.signal.find_peaks(d, threshold=0.5)[0]

In [ ]:
plt.plot(d)

In [ ]:
plt.imshow(np.diff(c, axis=0))

In [ ]:
display_image(get_crop(imgs[20], trenches2, 10).T, scale=True)

In [ ]:
hv.HoloMap({t: ui.RevImage(get_crop(imgs[t], trenches2, 10).T) for t in range(225)})